In [13]:
# --- SETUP ---
# Cài đặt thư viện cần thiết (nếu chưa có)
!pip install openpyxl pandas numpy --quiet

import pandas as pd
import numpy as np

# --- LOAD DATA ---
# Đọc file dữ liệu (bạn cần upload đúng tên file lên Colab trước)
price_raw = pd.read_excel('/content/Giá Nguyên Liệu.xlsx')
steel_finance = pd.read_excel('/content/bao_cao_tai_chinh.xlsx')
# macro_raw = pd.read_excel('/content/Dữ liêu vĩ mô.xlsx')  # nếu có thêm dữ liệu vĩ mô

# --- HÀM CHUẨN HÓA NGÀY ---
def fix_date(df):
    """
    Chuẩn hóa dữ liệu:
    - Tự động nhận diện hoặc tạo cột 'Date'
    - Chuyển về kiểu datetime
    - Loại bỏ lỗi thời gian
    """
    date_cols = [c for c in df.columns if isinstance(c, str) and ('date' in c.lower() or 'time' in c.lower())]
    if len(date_cols) == 0:
        df['Date'] = pd.to_datetime(df.iloc[:, 0], errors='coerce')
        df = df.drop(df.columns[0], axis=1)
    else:
        df['Date'] = pd.to_datetime(df[date_cols[0]], errors='coerce')
        if date_cols[0] != 'Date':
            df = df.drop(columns=[date_cols[0]])
    df = df.dropna(subset=['Date'])
    return df[['Date'] + [c for c in df.columns if c != 'Date']]

# --- CHUẨN HÓA TỪNG FILE ---
price = fix_date(price_raw)
steel = fix_date(steel_finance)
# macro = fix_date(macro_raw)  # nếu có thêm

# --- GỘP THEO NGÀY ---
def merge_daily(*dfs):
    """
    Hợp nhất nhiều dataframe theo cột 'Date' (kiểu outer join)
    """
    merged = dfs[0].set_index('Date').sort_index()
    for df in dfs[1:]:
        merged = merged.join(df.set_index('Date'), how='outer', rsuffix='_r')
    merged = merged.sort_index()
    return merged

merged = merge_daily(price, steel)
# merged = merge_daily(price, steel, macro)  # nếu có thêm dữ liệu

# --- KẾT QUẢ ---
print("Kích thước dữ liệu sau khi merge theo ngày:", merged.shape)
display(merged)


Kích thước dữ liệu sau khi merge theo ngày: (436, 51)


,Đơn vị: USD,% Thay đổi,Lần cuối,1,USD,=,26301.5,VND,CP,Năm,...,Vay và nợ thuê tài chính dài hạn (đồng),Người mua trả tiền trước ngắn hạn (đồng),Vay và nợ thuê tài chính ngắn hạn (đồng),Lợi thế thương mại (đồng),Trả trước dài hạn (đồng),Tài sản dài hạn khác (đồng),Phải thu dài hạn khác (đồng),Phải thu dài hạn (đồng),symbol,Các quỹ khác
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HPG,2025.0,...,2.835610e+13,6.192051e+11,6.848238e+13,5.209775e+10,4.390537e+12,0.000000e+00,8.255626e+11,8.497883e+11,HPG,NaN
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NKG,2025.0,...,2.048902e+12,1.029647e+11,4.907419e+12,NaN,5.605344e+11,NaN,1.337705e+09,1.337705e+09,NKG,NaN
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HSG,2025.0,...,0.000000e+00,2.218370e+11,4.403866e+12,0.000000e+00,2.629033e+11,0.000000e+00,6.803117e+11,1.930312e+12,HSG,NaN
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POM,2025.0,...,7.117350e+11,5.076675e+10,5.237683e+12,NaN,1.515004e+11,1.140299e+10,2.721977e+11,2.721977e+11,POM,NaN
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TVN,2025.0,...,1.833582e+12,2.781126e+11,7.779505e+12,2.341012e+10,3.775106e+11,4.560359e+11,4.759556e+10,6.864217e+10,TVN,1.469135e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-01,99.12,0.0492,2607004.680,vnd,/tấn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-08-01,101.81,0.0271,2677755.715,vnd,/tấn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-09-01,105.29,0.0342,2769284.935,vnd,/tấn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def fix_date(df):
    date_cols = [c for c in df.columns if isinstance(c, str) and ('date' in c.lower() or 'time' in c.lower())]
    if len(date_cols) == 0:
        # Giả định cột đầu tiên là ngày, convert
        df['Date'] = pd.to_datetime(df.iloc[:, 0], errors='coerce', dayfirst=True)
        df = df.drop(df.columns[0], axis=1)
    else:
        df['Date'] = pd.to_datetime(df[date_cols[0]], errors='coerce', dayfirst=True)
        if date_cols[0] != 'Date':
            df = df.drop(columns=[date_cols[0]])
    df = df.dropna(subset=['Date'])
    df = df.rename(columns=lambda x: x.strip())  # loại bỏ khoảng trắng dư
    return df[['Date'] + [c for c in df.columns if c != 'Date']]


In [15]:
def add_prefix(df, prefix):
    df = df.copy()
    df.columns = ['Date'] + [f"{prefix}_{c}" for c in df.columns if c != 'Date']
    return df

price = add_prefix(price, 'price')
steel = add_prefix(steel, 'steel')
merged = merge_daily(price, steel)


In [16]:
merged.isna().sum()


,0
price_Đơn vị: USD,255
price_% Thay đổi,255
price_Lần cuối,317
price_1,317
price_USD,317
price_=,436
price_26301.5,436
price_VND,436
steel_CP,181
steel_Năm,181


In [17]:
merged = merged.fillna(method='ffill')  # fill forward
merged = merged.fillna(method='bfill')  # fill backward


/tmp/ipython-input-3686621709.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged = merged.fillna(method='ffill')  # fill forward
/tmp/ipython-input-3686621709.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged = merged.fillna(method='bfill')  # fill backward


In [18]:
print("Kích thước dữ liệu sau khi merge theo ngày:", merged.shape)
display(merged)

Kích thước dữ liệu sau khi merge theo ngày: (436, 51)


,price_Đơn vị: USD,price_% Thay đổi,price_Lần cuối,price_1,price_USD,price_=,price_26301.5,price_VND,steel_CP,steel_Năm,...,steel_Vay và nợ thuê tài chính dài hạn (đồng),steel_Người mua trả tiền trước ngắn hạn (đồng),steel_Vay và nợ thuê tài chính ngắn hạn (đồng),steel_Lợi thế thương mại (đồng),steel_Trả trước dài hạn (đồng),steel_Tài sản dài hạn khác (đồng),steel_Phải thu dài hạn khác (đồng),steel_Phải thu dài hạn (đồng),steel_symbol,steel_Các quỹ khác
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-01,160.50,0.0881,1082832.755,vnd,/tấn,NaN,NaN,NaN,HPG,2025.0,...,2.835610e+13,6.192051e+11,6.848238e+13,5.209775e+10,4.390537e+12,0.000000e+00,8.255626e+11,8.497883e+11,HPG,1.469135e+11
1970-01-01,160.50,0.0881,1082832.755,vnd,/tấn,NaN,NaN,NaN,NKG,2025.0,...,2.048902e+12,1.029647e+11,4.907419e+12,5.209775e+10,5.605344e+11,0.000000e+00,1.337705e+09,1.337705e+09,NKG,1.469135e+11
1970-01-01,160.50,0.0881,1082832.755,vnd,/tấn,NaN,NaN,NaN,HSG,2025.0,...,0.000000e+00,2.218370e+11,4.403866e+12,0.000000e+00,2.629033e+11,0.000000e+00,6.803117e+11,1.930312e+12,HSG,1.469135e+11
1970-01-01,160.50,0.0881,1082832.755,vnd,/tấn,NaN,NaN,NaN,POM,2025.0,...,7.117350e+11,5.076675e+10,5.237683e+12,0.000000e+00,1.515004e+11,1.140299e+10,2.721977e+11,2.721977e+11,POM,1.469135e+11
1970-01-01,160.50,0.0881,1082832.755,vnd,/tấn,NaN,NaN,NaN,TVN,2025.0,...,1.833582e+12,2.781126e+11,7.779505e+12,2.341012e+10,3.775106e+11,4.560359e+11,4.759556e+10,6.864217e+10,TVN,1.469135e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-01,99.12,0.0492,2607004.680,vnd,/tấn,NaN,NaN,NaN,TVN,2013.0,...,3.930311e+12,3.759772e+10,8.049557e+12,0.000000e+00,8.227411e+11,5.831473e+11,2.340069e+08,6.360428e+09,TVN,8.125776e+09
2025-08-01,101.81,0.0271,2677755.715,vnd,/tấn,NaN,NaN,NaN,TVN,2013.0,...,3.930311e+12,3.759772e+10,8.049557e+12,0.000000e+00,8.227411e+11,5.831473e+11,2.340069e+08,6.360428e+09,TVN,8.125776e+09
2025-09-01,105.29,0.0342,2769284.935,vnd,/tấn,NaN,NaN,NaN,TVN,2013.0,...,3.930311e+12,3.759772e+10,8.049557e+12,0.000000e+00,8.227411e+11,5.831473e+11,2.340069e+08,6.360428e+09,TVN,8.125776e+09
